# Xgboost with Ray, Let's see what's the differecne

In [1]:
import ray
ray.init()

2021-12-26 17:06:50,204	INFO services.py:1340 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.84.193',
 'raylet_ip_address': '192.168.84.193',
 'redis_address': '192.168.84.193:6379',
 'object_store_address': '/tmp/ray/session_2021-12-26_17-06-47_293113_25355/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-12-26_17-06-47_293113_25355/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-12-26_17-06-47_293113_25355',
 'metrics_export_port': 62230,
 'node_id': '77a22e7ce284db5bf6a47f9e16492a40cd97437e9bb805078349602d'}

## Check memory first, It shows noting there

In [2]:
!ray memory

======== Object references status: 2021-12-26 12:56:41.140323 ========
Grouping by node address...        Sorting by object size...        Display allentries per group...


To record callsite information for each ObjectRef created, set env variable RAY_record_ref_creation_sites=1

--- Aggregate object store stats across all nodes ---
Plasma memory usage 0 MiB, 0 objects, 0.0% full, 0.0% needed



## Train Xgboost with ray , is so easy

In [2]:
# prepare dataset
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

train_x, train_y = load_breast_cancer(return_X_y=True)
train_set = RayDMatrix(train_x, train_y)  # <- major difference


### you can see that while creating RayDMartix, nothing happend in Ray cluster

In [5]:
!ray memory

======== Object references status: 2021-12-26 12:57:33.942124 ========
Grouping by node address...        Sorting by object size...        Display allentries per group...


To record callsite information for each ObjectRef created, set env variable RAY_record_ref_creation_sites=1

--- Aggregate object store stats across all nodes ---
Plasma memory usage 0 MiB, 0 objects, 0.0% full, 0.0% needed



In [6]:
# train and save
evals_result = {}
bst = train(
    {
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
    },
    train_set,
    evals_result=evals_result,
    evals=[(train_set, "train")],
    verbose_eval=False,
    ray_params=RayParams(
        num_actors=2,  # Number of remote actors
        cpus_per_actor=1))

bst.save_model("model.xgb")
print("Final training error: {:.4f}".format(
    evals_result["train"]["error"][-1]))

2021-12-26 12:57:39,135	INFO main.py:971 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
2021-12-26 12:57:40,152	INFO main.py:1016 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=636) [12:57:40] task [xgboost.ray]:140303809745104 got new rank 0
(_RemoteRayXGBoostActor pid=639) [12:57:40] task [xgboost.ray]:140088995584592 got new rank 1
2021-12-26 12:57:42,445	INFO main.py:1498 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 3.39 seconds (2.28 pure XGBoost training time).


Final training error: 0.0053


### once train the model, the RayDMatrix changed

In [11]:
print(f"original data shape: {train_x.shape}")
for i in [0, 1]:
    df = ray.get(train_set.refs[0]['data'])
    print(f"part: {i}", df.shape, type(df))

original data shape: (569, 30)
part: 0 (285, 30)
part: 1 (285, 30)


In [12]:
!ray memory

======== Object references status: 2021-12-26 13:01:13.862222 ========
Grouping by node address...        Sorting by object size...        Display allentries per group...


--- Summary for node address: 192.168.84.193 ---
Mem Used by Objects  Local References  Pinned        Pending Tasks  Captured in Objects  Actor Handles
158678.0 B           11, (86905.0 B)   1, (71773.0 B)  0, (0.0 B)     0, (0.0 B)           0, (0.0 B)   

--- Object references for node address: 192.168.84.193 ---
IP Address | PID | Type | Call Site | Size | Reference Type | Object Ref

192.168.84.193 | 482 | Driver |           | 15.0 B | LOCAL_REFERENCE | ffffffffffffffffffffffffffffffffffffffff0100000005000000


192.168.84.193 | 482 | Driver |           | 15.0 B | LOCAL_REFERENCE | ffffffffffffffffffffffffffffffffffffffff010000000b000000


192.168.84.193 | 482 | Driver |           | 15.0 B | LOCAL_REFERENCE | ffffffffffffffffffffffffffffffffffffffff0100000004000000


192.168.84.193 | 482 | Driver |           | 15

## performance check

In [3]:
import numpy as np

train_x = np.random.rand(100000, 200)
train_y = np.random.rand(100000) > 0.5

In [4]:
%%time
train_set = RayDMatrix(train_x, train_y)

evals_result = {}
bst = train(
    {
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
        'seed': 200,
    },
    
    train_set,
    num_boost_round=300,
    evals_result=evals_result,
    evals=[(train_set, "train")],
    verbose_eval=False,
    ray_params=RayParams(
        num_actors=2,  # Number of remote actors
        cpus_per_actor=4))

bst.save_model("model.xgb")
print("Final training error: {:.4f}".format(
    evals_result["train"]["error"][-1]))

2021-12-26 17:10:21,582	INFO main.py:971 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
2021-12-26 17:10:22,943	INFO main.py:1016 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=25502) [17:10:23] task [xgboost.ray]:140203909791952 got new rank 0
(_RemoteRayXGBoostActor pid=25510) [17:10:23] task [xgboost.ray]:139989723807312 got new rank 1
2021-12-26 17:10:53,594	INFO main.py:1091 -- Training in progress (30 seconds since last restart).
2021-12-26 17:11:23,809	INFO main.py:1091 -- Training in progress (61 seconds since last restart).
2021-12-26 17:11:53,975	INFO main.py:1091 -- Training in progress (91 seconds since last restart).
2021-12-26 17:12:24,157	INFO main.py:1091 -- Training in progress (121 seconds since last restart).
2021-12-26 17:12:54,360	INFO main.py:1091 -- Training in progress (151 seconds since last restart).
2021-12-26 17:13:24,539	INFO main.py:1091 -- Training in progress (181 seconds since 

Final training error: 0.0275
CPU times: user 8.54 s, sys: 2.34 s, total: 10.9 s
Wall time: 4min 20s


In [9]:
train_set.refs

{0: {'data': ObjectRef(ffffffffffffffffffffffffffffffffffffffff0100000015000000),
  'label': ObjectRef(ffffffffffffffffffffffffffffffffffffffff0100000016000000),
  'weight': ObjectRef(ffffffffffffffffffffffffffffffffffffffff0100000017000000),
  'base_margin': ObjectRef(ffffffffffffffffffffffffffffffffffffffff0100000018000000),
  'label_lower_bound': ObjectRef(ffffffffffffffffffffffffffffffffffffffff0100000019000000),
  'label_upper_bound': ObjectRef(ffffffffffffffffffffffffffffffffffffffff010000001a000000)},
 1: {'data': ObjectRef(ffffffffffffffffffffffffffffffffffffffff010000001b000000),
  'label': ObjectRef(ffffffffffffffffffffffffffffffffffffffff010000001c000000),
  'weight': ObjectRef(ffffffffffffffffffffffffffffffffffffffff010000001d000000),
  'base_margin': ObjectRef(ffffffffffffffffffffffffffffffffffffffff010000001e000000),
  'label_lower_bound': ObjectRef(ffffffffffffffffffffffffffffffffffffffff010000001f000000),
  'label_upper_bound': ObjectRef(ffffffffffffffffffffffffffffffff

In [10]:
print(f"original data shape: {train_x.shape}")
for i in [0, 1]:
    df = ray.get(train_set.refs[0]['data'])
    print(f"part: {i}", df.shape, type(df))

original data shape: (10000, 200)
part: 0 (5000, 200) <class 'pandas.core.frame.DataFrame'>
part: 1 (5000, 200) <class 'pandas.core.frame.DataFrame'>


In [16]:
!ray memory

======== Object references status: 2021-12-26 13:11:49.156898 ========
Grouping by node address...        Sorting by object size...        Display allentries per group...


--- Summary for node address: 192.168.84.193 ---
Mem Used by Objects  Local References  Pinned        Pending Tasks  Captured in Objects  Actor Handles
162664906.0 B        23, (82262699.0 B)  1, (80402207.0 B)  0, (0.0 B)     0, (0.0 B)           0, (0.0 B)   

--- Object references for node address: 192.168.84.193 ---
IP Address | PID | Type | Call Site | Size | Reference Type | Object Ref

192.168.84.193 | 482 | Driver |           | 15.0 B | LOCAL_REFERENCE | ffffffffffffffffffffffffffffffffffffffff0100000018000000


192.168.84.193 | 482 | Driver |           | 15.0 B | LOCAL_REFERENCE | ffffffffffffffffffffffffffffffffffffffff0100000005000000


192.168.84.193 | 482 | Driver |           | 15.0 B | LOCAL_REFERENCE | ffffffffffffffffffffffffffffffffffffffff010000001a000000


192.168.84.193 | 482 | Driver |          

In [17]:
# check perfromance for original xgb
import xgboost as xgb

In [24]:
%%time
# read in data
dtrain = xgb.DMatrix(train_x, train_y)

evals_result = {}

bst = xgb.train(
    {
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
        'seed': 200,
    },
    dtrain,
    num_boost_round=300,
    evals_result=evals_result,
    evals=[(dtrain, "train")],
    verbose_eval=False
)
bst.save_model("model.xgb")
print("Final training error: {:.4f}".format(
    evals_result["train"]["error"][-1]))


Final training error: 0.0877
CPU times: user 53min 44s, sys: 4.89 s, total: 53min 49s
Wall time: 7min 38s
